### Rule-Based Aspect Extraction and Sentiment Analysis
Using part-of-speech tagging and grammatical dependencies (spaCy library). Done by Anood

In [1]:
import spacy
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_sm')

2023-03-09 11:38:47.338220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Rule 1: Extract adjective-noun pairs (where the adjective directly precedes the noun).
Output for each sentence is {'aspect term': [list of adjectives]}

In [2]:
sentences1 = ['The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers',
            'This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.',
            'Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders.',
             'Mr. Baldwin stated the deficient performance was caused by maintenance procedural inadequacy which allowed work to proceed with the relay energized.'
            ]

extracted_aspects = []

for sentence in sentences1:
    doc = nlp(sentence)
    noun_adj_pairs = defaultdict(dict)
    for token in doc:
        adj = []
        noun = ""
        if token.pos_ == 'NOUN':
            for child in token.children:
                if child.pos_ == 'ADJ':
                    noun = token.text
                    adj.append(child)
        if noun and adj:
            noun_adj_pairs[sentence].update({noun:adj})
#     print(sentence)
#     print(noun_adj_pairs, "\n")
    if len(noun_adj_pairs) != 0:
        extracted_aspects.append(dict(noun_adj_pairs))

extracted_aspects


[{'The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers': {'guidance': [Inadequate,
    procedural],
   'coordination': [unclear],
   'proceduers': [applicable]}},
 {'This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.': {'environment': [human,
    likely],
   'communication': [poor],
   'reliance': [much]}},
 {'Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders.': {'trend': [negative]}},
 {'Mr. Baldwin stated the deficient performance was caused by maintenance procedural inadequacy which allowed work to proceed with the relay energized.': {'performance': [d

In [3]:
#Visualizing dependencies
displacy.render(nlp(sentences1[0]), style='dep', jupyter=True)

#### Rule 2: Extract noun-adjective pairs (where the adjective comes after the noun)
Output for each sentence is {'aspect term': [list of adjectives]}

In [4]:
sentences2 =['Operator training was also poor with respect to how to cope with the conditions with which the operators at Fukushima were faced and availability of portable equipment was inadequate.',
            'This backlog of orders is reported on a weekly basis and was rated as Yellow, or deficient, in 70 of the 78 weeks between June 2012 and December 2013.',
            'the Plant Health Committee meeting the Appendix R Program Manager reported that this fire protection program health was Red'
           ]

for sentence in sentences2:
    doc = nlp(sentence)
    noun_adj_pairs = defaultdict(dict)
    for token in doc:
        adj = []
        noun = ""
        if (token.dep_ == 'ROOT') or (token.dep_ == 'conj') or (token.pos_ == 'AUX'):
            for child in token.children:
                if (child.pos_ == 'NOUN') or (child.pos == 'PROPN'):
                    noun = child.text
                    continue
                if (child.pos_ == 'ADJ') or (child.text in ['Red','Yellow']):
                    adj.append(child.text)
        if noun and adj:
            noun_adj_pairs[sentence].update({noun:adj})
    if len(noun_adj_pairs) != 0:
#         print(sentence)
        print(dict(noun_adj_pairs), '\n')
        if noun_adj_pairs not in extracted_aspects:
            extracted_aspects.append(dict(noun_adj_pairs))
    
# noun_adj_pairs
# extracted_aspects


{'Operator training was also poor with respect to how to cope with the conditions with which the operators at Fukushima were faced and availability of portable equipment was inadequate.': {'training': ['poor'], 'availability': ['inadequate']}} 

{'the Plant Health Committee meeting the Appendix R Program Manager reported that this fire protection program health was Red': {'health': ['Red']}} 



In [5]:
displacy.render(nlp(sentences2[0]), style='dep', jupyter=True)

#### Rule 3: Extracting aspect terms that are mentioned after "lack of"

In [6]:
sentences3 = ['This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.',
             'Other failures were due to lack of or inadequate regular inspections, maintenance (cleaning) or replacement and due to lack of a thorough understanding by operators of the infrequently used system.',
             'Mr. Wardell replied that the root cause of the safety system functional failure was due to lack of clear standards for risk.'
            ]
             
for sentence in sentences3:
    doc = nlp(sentence)
    noun_adj_pairs = defaultdict(dict)
    for token in doc:
        adj = []
        noun = ""
        if ((token.head.text).lower() == 'lack'):
            for child in token.children:
                if (child.pos_ == 'NOUN'):
                    noun = child.text
                    adj.append(token.head.text)
        if noun and adj:
            noun_adj_pairs[sentence].update({noun:adj})
#     print(sentence)
    print(dict(noun_adj_pairs), '\n')
    if len(noun_adj_pairs) != 0:
        extracted_aspects.append(noun_adj_pairs)
        

{'This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.': {'leadership': ['lack']}} 

{'Other failures were due to lack of or inadequate regular inspections, maintenance (cleaning) or replacement and due to lack of a thorough understanding by operators of the infrequently used system.': {'maintenance': ['lack'], 'understanding': ['lack']}} 

{'Mr. Wardell replied that the root cause of the safety system functional failure was due to lack of clear standards for risk.': {'standards': ['lack']}} 



In [7]:
displacy.render(nlp(sentences3[-1]), style='dep', jupyter=True)

#### Total list of extracted aspects using all three rules above

In [8]:
for sentence in extracted_aspects:
    for k, v in sentence.items():
        print("Sentence:   ", k)
        print("Extracted Aspects:   ", v)
        print('\n')

Sentence:    The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers
Extracted Aspects:    {'guidance': [Inadequate, procedural], 'coordination': [unclear], 'proceduers': [applicable]}


Sentence:    This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.
Extracted Aspects:    {'environment': [human, likely], 'communication': [poor], 'reliance': [much]}


Sentence:    Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders.
Extracted Aspects:    {'trend': [negative]}


Sentence:    Mr. Baldwin stated the deficient performance was caused by maintenance procedural inadequ

#### Sentiment Analysis of extracted aspects

Download opinion lexicon (a list of positive/negative words) from: https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#:~:text=Opinion%20Lexicon%3A%20A%20list%20of,Liu%2C%20KDD%2D2004

In [9]:
with open('positive-words.txt') as f:
    lines = f.read().splitlines() 
    pos_words = [x for x in lines if not x.startswith(';')][1:]

with open('negative-words.txt',encoding = "ISO-8859-1") as f:
    lines = f.read().splitlines() 
    neg_words = [x for x in lines if not x.startswith(';')][1:]

neg_aspects = []
for sentence in extracted_aspects:
    for key,values in sentence.items():
        for aspect, opinions in values.items():
            for adj in opinions:
                adj = str(adj).lower()
                if adj in neg_words:
                    if aspect not in neg_aspects:
                        neg_aspects.append(aspect)
                    print('sentence:', key)
                    print('aspect term:', aspect)
                    print('sentiment:', 'negative')
                    print('safety traits:', [], '\n')

sentence: The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers
aspect term: guidance
sentiment: negative
safety traits: [] 

sentence: The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers
aspect term: coordination
sentiment: negative
safety traits: [] 

sentence: This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.
aspect term: communication
sentiment: negative
safety traits: [] 

sentence: Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders.
aspect term:

In [24]:
print("List of negatively-described aspects:")
neg_aspects

List of negatively-described aspects:


['guidance',
 'coordination',
 'communication',
 'trend',
 'performance',
 'training',
 'availability',
 'leadership',
 'maintenance',
 'understanding',
 'standards']

#### Mapping negative aspects to safety traits using seed words and Word2Vec cosine similarity.

In [11]:
personal_accountability = ['responsible', 'encourage', 'accountable', 'help', 'ownership', 'support', 'trained', 'qualified', 'understand', 'non-cited', 'involvement', 'self-identifying']
questioning_attitude = ['complacent', 'discrepancy', 'anomaly', 'assumption', 'question', 'uncertain', 'unknown', 'risk', 'investigate', 'unexpected', 'unclear', 'review', 'degrading', 'aging']
safety_communication = ['communication', 'information', 'safety', 'share', 'respond', 'listen', 'concerns', 'expectations', 'clear']
leadership_values = ['leadership', 'management', 'leaders', 'owner', 'guidance', 'decisions', 'policies', 'resources', 'staffing', 'expectations', 'oversight', 'reinforce', 'decision-making', 'mentoring', 'coaching', 'encourage', 'incentive', 'reward', 'priorities', 'plan', 'delegate', 'define', 'manage', 'support', 'resolve', 'motivate', 'address', 'resources', 'translate', 'funding', 'implementation']
decision_making = ['decision-making', 'thorough', 'conservative', 'systematic', 'consistent', 'process', 'leaders', 'choices', 'consequences', 'reinforce', 'accountable', 'authority', 'responsibility']
respectful_environment = ['trust', 'respect', 'encourage', 'opinions', 'bullying', 'fair', 'productive', 'concerns', 'voice', 'discussions', 'value', 'insight', 'perspectives', 'collaboration', 'conflict', 'listening']
continuous_learning = ['learn', 'training', 'self-assessments', 'improve', 'performance', 'monitor', 'experience', 'assessment', 'corrective action', 'benchmarking', 'knowledge', 'competence', 'skills', 'develop', 'understand']
problem_identification = ['identify', 'address', 'correct', 'corrective action', 'issues', 'deviation', 'degraded conditions', 'evaluation', 'problems', 'root cause', 'investigation', 'recommendation', 'cause analysis', 'resolution', 'mitigate', 'trends']
raising_concerns = ['concerns', 'confidence', 'address', 'feedback', 'listening', 'report']
work_processes = ['plan', 'control', 'activities', 'production', 'schedule', 'manage', 'execute', 'risk', 'coordinate', 'margins', 'operate', 'maintain', 'maintenance', 'document', 'complete', 'procedures', 'packages', 'accurate', 'current', 'backlog', 'follow', 'review', 'instructions', 'operations', 'design', 'requirements', 'operations']

safety_traits = ['Personal Accountability', 'Questioning Attitue', 'Effective Safety Communication', 
                 'Leadership Safety Values and Actions','Decision Making', 'Respectful Work Environment', 
                 'Continuous Learning','Problem Identification and Resolution','Environment for Raising Concerns',
                 'Work Processes']

safety_categories = [personal_accountability, questioning_attitude, safety_communication, leadership_values,
                    decision_making, respectful_environment, continuous_learning, problem_identification,
                    raising_concerns, work_processes]

In [12]:
from gensim.models import Word2Vec
import gensim.downloader

model = gensim.downloader.load('word2vec-google-news-300')

In [23]:
aspect_categories = defaultdict(list)
for sentence in extracted_aspects:
    for key,values in sentence.items():
        for aspect, opinions in values.items():
            if aspect not in neg_aspects:
                continue
            max_similarity = 0
            i = -1
            for category in safety_categories:
                i+= 1
                for seed_word in category:
                    try:
                        similarity = model.similarity(aspect, seed_word)
                        if similarity >= max_similarity:
                            max_similarity = similarity 
                            trait = safety_traits[i]
                    except:
                        continue
            if trait not in aspect_categories:
                aspect_categories[key].append((aspect,trait))
                
for k, v in aspect_categories.items():
    print("Sentence:   ", k)
    print("Extracted Aspects:   ", [x[0] for x in v])
    print("Safety traits:   ", [x[1] for x in v])
    print("\n")

Sentence:    The root causes were identified as follows: Root Cause for Sub-Problem 1: Inadequate procedural guidance and unclear coordination between applicable proceduers
Extracted Aspects:    ['guidance', 'coordination']
Safety traits:    ['Leadership Safety Values and Actions', 'Work Processes']


Sentence:    This ultimately created an environment that promulgated a human error-likely environment.” More specifically, the RCE team determined that the environment consisted of poor communication, lack of engineering leadership, too much reliance on vendor designs, time pressure, and distractions.
Extracted Aspects:    ['communication', 'leadership']
Safety traits:    ['Effective Safety Communication', 'Leadership Safety Values and Actions']


Sentence:    Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders.
Extracted Aspects:    ['trend']
Safety traits:    ['Problem Identification and Resolution

#### Aspects extraction for all sentences (need to define more rules), below are just random tests

In [17]:
import json

def rule1(doc):
    noun_adj_pairs = {}
    for token in doc:
        adj = []
        noun = ""
        if token.pos_ == 'NOUN':
            for child in token.children:
                if child.pos_ == 'ADJ':
                    noun = token.text
                    adj.append(child)
        if noun and adj:
            noun_adj_pairs.update({noun:adj})
    if len(noun_adj_pairs) != 0:
        return noun_adj_pairs   

def rule2(doc):
    noun_adj_pairs = {}
    for token in doc:
        adj = []
        noun = ""
        if (token.dep_ == 'ROOT') or (token.dep_ == 'conj') or (token.pos_ == 'AUX'):
            for child in token.children:
                if (child.pos_ == 'NOUN') or (child.pos == 'PROPN'):
                    noun = child.text
                    continue
                if (child.pos_ == 'ADJ') or (child.text in ['Red','Yellow']):
                    adj.append(child.text)
        if noun and adj:
            noun_adj_pairs.update({noun:adj})
    if len(noun_adj_pairs) != 0:
        return noun_adj_pairs

def rule3(doc):
    noun_adj_pairs = {}
    for token in doc:
        adj = []
        noun = ""
        if ((token.head.text).lower() == 'lack'):
            for child in token.children:
                if (child.pos_ == 'NOUN'):
                    noun = child.text
                    adj.append(token.head.text)
        if noun and adj:
            noun_adj_pairs.update({noun:adj})
    if len(noun_adj_pairs) != 0:
        return noun_adj_pairs   

json_file = open('extracted__neg_sentences.json', 'r')
listt = json.load(json_file)
for dictt in listt:
    sentence = dictt['sentence']
    doc = nlp(sentence)
    if rule1(doc) != None:
        print(sentence,'\n', rule1(doc))
    if rule2(doc) != None:
        print(rule2(doc))
    if rule3(doc) != None:
        print(rule3(doc))
    print('\n')

DCPP acted prompt with corrective actions and submitted a Licensee Event Report when it discovered Technical Specification non-compliance on the Low Temperature Overpressure Protection System. 
 {'actions': [corrective]}




Three Station Level Human Performance Event Clock Resets occurred during the fourth quarter of 2013, causing the station’s 18-month indicator for such Resets to become Yellow (deficient). 
 {'quarter': [fourth]}


Operations performance with respect to human error rate has been Red (Unsatisfactory) since July 2013. 
 {'rate': [human]}
{'performance': ['Red']}




Problems have been mostly due to age-related issues and lack of adequate inspection, maintenance, and component replacement, especially electrical contacts. 
 {'inspection': [adequate], 'contacts': [electrical]}
{'issues': ['due']}
{'contacts': ['lack']}


problems and failures increased the frequency of Critical Event Clock Resets in the second half of 2013. 
 {'half': [second]}


Also, equipment problems

Contromatics and Bettis actuators are experiencing obsolescence problems in spare parts availability. 
 {'parts': [spare]}


Several AOVs are not controlling well, and action plans are being developed. 
 {'AOVs': [Several]}


If the vessel, which normally operates at approximately 600 degrees F and 2200 pounds per square inch of pressure (psi), were to experience a cold-water shock from inadvertently injecting cold water into the vessel while at operating pressure, it is possible that existing cracks in the vessel could rapidly enlarge, resulting in a vessel rupture. 
 {'inch': [square], 'water': [cold]}


{'rupture': ['difficult']}


The performance summary included both the original piping and additional bypass piping, which was installed due to a concern that localized corrosion was occurring in the portion of the piping buried below sea level in the tidal zone outside the intake structure. 
 {'piping': [additional], 'zone': [tidal], 'structure': [intake]}
{'bypass': ['additional']}





Problems have been mostly due to age-related issues and lack of adequate inspection, maintenance, and component replacement, especially electrical contacts. 
 {'inspection': [adequate], 'contacts': [electrical]}
{'issues': ['due']}
{'contacts': ['lack']}


Equipment problems and failures increased the frequency of Critical Event Clock Resets in the second half of 2013. 
 {'half': [second]}


Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders. 
 {'trend': [negative]}


Systems that are the sources of emergency electrical power to the station’s vital electrical equipment, the station’s Emergency Diesel Generators and the 230 kV system that is supplied from the offsite electrical grid, were found to be operational but have been a focus of station and NRC attention. 
 {'power': [electrical], 'equipment': [vital, electrical], 'grid': [offsite, electrical]}


Excessive Critical Component Failure/Ad

Dr. Budnitz reported that he had a potential scheduling conflict with the date of the next meeting of the State Water Resources Control Board’s Review Committee for Nuclear Fueled Power Plants (RCNFPP) which is now scheduled in Sacramento, California, on Monday, November 4, 2013. 
 {'conflict': [potential], 'meeting': [next]}


Mr. Wardell reported the system health status was affected by the fact there is a new system/program owner and long-range planning has not been completed. 
 {'owner': [new], 'range': [long]}


Refueling Equipment Reliability – Mr. Wardell reported reliability issues focused mostly on cranes and moveable bridges due to aging components and exposure to humidity. 
 {'bridges': [moveable]}


The program health is in Yellow status principally due to a new program manager. 
 {'status': [Yellow], 'manager': [new]}
{'health': ['due']}


U-2 Reactor Coolant Pump Seal Failure Root Cause Evaluation – the U-2 Reactor Coolant Pump 2-2 seals triggered high alarms on seal disc

On this occasion the PHC reviewed the Appendix R Fire Protection Program which was in Red status due to outstanding Nonconformance Reports (NCR) and NRC Findings and Mr. Wardell reported the backlog for these items has been reduced and the system is expected to return to White status by the next PHC meeting. 
 {'status': [White], 'meeting': [next]}


The PHC also reviewed Critical Component Failures in the Fire Protection System including 16 inoperable fire doors which has required the plant to post fire watches in the affected areas. 
 {'doors': [inoperable], 'areas': [affected]}


The PHC also reviewed the Radiation Monitoring System which is in White status for Unit-1 (U-1) and in Yellow status for Unit-2 (U-2) because of equipment reliability problems due to aging. 
 {'status': [Yellow], 'Unit-2': [U-2]}


He further observed that the matter of the Containment equipment hatch seismic qualification not having been identified during thirty years of DCPP’s operation was troubling to h

A root cause evaluation was performed and lack of guidance for highlighting significant first-time complex high risk projects was identified as a cause due to these projects not having been addressed through the project management process. 
 {'time': [first], 'risk': [high], 'projects': [significant, complex]}
{'guidance': ['lack']}


The DCISC fact-finding team met with DCPP representatives to review PG&E’s concerns and the possible effect on the plant including the potential for increased salt deposition issues with insulators and flashover events and the higher risk of loss of offsite power, discussed previously by Mr. Linnen, and for increased corrosion of equipment due to salt. 
 {'effect': [possible], 'events': [flashover], 'risk': [higher], 'power': [offsite]}


He stated that as a result of the closure of SONGS many people are going to get sick and die. 
 {'people': [many]}


Dr. Budnitz, in response to Ms. Becker’s comment, stated the principal issue identified to date with th

Other failures were due to lack of or inadequate regular inspections, maintenance (cleaning) or replacement and due to lack of a thorough understanding by operators of the infrequently used system. 
 {'failures': [Other], 'inspections': [inadequate, regular], 'maintenance': [due], 'understanding': [thorough]}
{'failures': ['due'], 'maintenance': ['inadequate', 'regular'], 'cleaning': ['due']}
{'maintenance': ['lack'], 'understanding': ['lack']}


Additionally, the Unit 1 mechanical Rod Cluster Control Assembly (RCCA) Change Tool is not working properly, and a repair or new one is needed in time for Outage 1R18. 
 {'one': [new]}


Conclusions: The DCPP Fuel Handling System has been problematic since Refueling Outage 2R14, and caused significant delays in Outage 2R17 fuel movement but no nuclear safety concerns. 
 {'delays': [significant], 'safety': [nuclear]}
{'Conclusions': ['problematic']}


The problems are mostly due to age-related issues and lack of adequate inspection, maintenance

There was considerable variety in the nature of the SSFFs. Some examples are listed below: Non-conservative Technical Specification (TS) First Level Undervoltage Relay (FLUR)/Second Level Undervoltage Relay (SLUR) results in loss of power to Emergency Diesel Generator (EDG) start instrumentation, Units 1 and 2 230kV allowed outage time exceeded when cross-tied between Units 1 and 2 Mode 3 Entry with AFW Pump 1-1 inoperable Auxiliary Building Ventilation System single failure, Units 1 and 2 Three Losses of Offsite Power during Refueling Outage 2R16, Unit 1 DCPP’s Root Cause Evaluation (RCE) Report of these SSFFs further notes that, beginning with the discovery of incorrect open limit switch settings on motor-operated Emergency Core Cooling System (ECCS) sump suction valves in 2009, “DCPP experienced multiple events that resulted in the loss of a system safety function to shut 
 {'variety': [considerable], 'failure': [inoperable, single], 'limit': [open], 'settings': [incorrect], 'events

Operations performance with respect to human error rate has been Red (Unsatisfactory) since July 2013. 
 {'rate': [human]}
{'performance': ['Red']}


Violation 480V Switchgear Room HVA.” It was determined that the two 24-inch diameter fans specified for the above purpose would, in fact, have overloaded their power supply. 
 {'purpose': [above]}
{'480V': ['Switchgear']}


However, these activities were terminated as a result of concerns by the public and local governmental officials that the soundings could have an adverse impact on sea life in that area. 
 {'officials': [public, governmental], 'impact': [adverse]}


Most recently a public interest group expressed concern over some issues related to aspects of DCPP’s operating license that are specifically related to seismicity. 
 {'interest': [public]}


The audit’s three findings pertaining to Corrective Action were as follows: Inappropriate use of Long-Term Corrective Actions (LTCA) and repeated extensions of due dates for Corrective



The momentary loss of power to the Unit 1 Spent Fuel Pool (SFP) Cooling Pump caused it to trip off. 
 {'loss': [momentary]}


DCPP’s Outage 1R18 results were positive with the one exception of temporary loss of the Unit 1 Spent Fuel Pool Cooling Pump due to an electric grid disturbance. 
 {'loss': [temporary], 'grid': [electric]}
{'results': ['positive']}


Salt Deposition The deposition of salt onto equipment can have negative impacts on reliability and on plant and personnel safety. 
 {'impacts': [negative]}


 Recent problems with salt deposition at DCPP include a series of flashover events occurring on high voltage bushings. 
 {'problems': [Recent], 'events': [flashover], 'voltage': [high]}


 A particular area of risk that required very careful evaluation will be loss of offsite power (LOOP) due to potential simultaneous electrical failures in the 230 kV and 500 kV switch yards. 
 {'area': [particular], 'evaluation': [careful], 'power': [offsite], 'failures': [potential, simulta